In [ ]:
import gzip
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import numpy as np
import torchvision.transforms as transforms
from matplotlib import pyplot as plt
from torch.utils.data import TensorDataset, DataLoader, Dataset
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets
from PIL import Image
import pandas as pd
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import silhouette_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report, confusion_matrix, f1_score, precision_score, recall_score

In [ ]:
# open the gzip files
train = gzip.open('train-images-idx3-ubyte.gz', 'rb')
train_label = gzip.open('train-labels-idx1-ubyte.gz', 'rb')
test = gzip.open('t10k-images-idx3-ubyte.gz', 'rb')
test_label = gzip.open('t10k-labels-idx1-ubyte.gz', 'rb')

# reading bytes from the gzip files
train.read(16)
train_label.read(8)
test.read(16)
test_label.read(8)


# load image data and reshape
train_image_data = np.frombuffer(train.read(), dtype=np.uint8).reshape(-1, 28, 28)
train_label_data = np.frombuffer(train_label.read(), dtype=np.uint8)

test_image_data = np.frombuffer(test.read(), dtype=np.uint8).reshape(-1, 28, 28)
test_label_data = np.frombuffer(test_label.read(), dtype=np.uint8)

# Ensure train_image_data and train_label_data have compatible shapes
train_image_data = train_image_data[:len(train_label_data)]

# train-validation split(80-20 split)
train_images, val_images, train_labels, val_labels = train_test_split(
    train_image_data, train_label_data, test_size=0.2, random_state=42
)

In [ ]:
# Visualize all 10 unique labels
plt.figure(figsize=(10, 5))
for i in range(10):
    index = np.where(train_labels == i)[0][0]
    plt.subplot(2, 5, i + 1)
    plt.imshow(train_images[index], cmap='gray')
    plt.title(f'Label: {i}')
    plt.axis('off')
plt.tight_layout()
plt.show()

0 : 'T-shirt'
1 : 'Trouser',
2 : 'Pullover',
3 : 'Dress',
4 : 'Coat',
5 : 'Sandal',
6 : 'Shirt',
7 : 'Sneaker',
8 : 'Bag',
9 : 'Ankle Boot'

In [ ]:
# This is for Exploratory Data Analysis

# Flatten the images
train_images_flat = train_image_data.reshape(train_image_data.shape[0], -1)
test_images_flat = test_image_data.reshape(test_image_data.shape[0], -1)

# Create DataFrames
train_df = pd.DataFrame(train_images_flat)
train_df['label'] = train_label_data

test_df = pd.DataFrame(test_images_flat)
test_df['label'] = test_label_data

print(train_df.shape)
print(test_df.shape)

In [ ]:
# Visualize class distribution in training data
plt.figure(figsize=(10, 5))
sns.countplot(x=train_df['label'])
plt.title('Training Data Class Distribution')
plt.show()

# Visualize class distribution in test data
plt.figure(figsize=(10, 5))
sns.countplot(x=test_df['label'])
plt.title('Test Data Class Distribution')
plt.show()

In [ ]:
# check if data has any nan values (Data Cleaning)
print(np.isnan(train_images).any())
print(np.isnan(val_images).any())
print(np.isnan(test_image_data).any())

In [ ]:
transform_train = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

transform_val_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

class CustomDataset(Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images.numpy().astype(np.uint8)
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        image = Image.fromarray(self.images[idx])
        if self.transform:
            image = self.transform(image)
        label = self.labels[idx]
        return image, label

# Convert data to PyTorch tensors
train_images = torch.tensor(train_images, dtype=torch.float32)
train_labels = torch.tensor(train_labels, dtype=torch.long)  
val_images = torch.tensor(val_images, dtype=torch.float32)
val_labels = torch.tensor(val_labels, dtype=torch.long)
test_image_data = torch.tensor(test_image_data, dtype=torch.float32)
test_label_data = torch.tensor(test_label_data, dtype=torch.long)

# Create custom datasets with transforms (Data transformation)
train_dataset = CustomDataset(train_images, train_labels, transform=transform_train)
val_dataset = CustomDataset(val_images, val_labels, transform=transform_val_test)
test_dataset = CustomDataset(test_image_data, test_label_data, transform=transform_val_test)

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

In [ ]:
# PCA for dimensionality reduction
pca = PCA(n_components=40)
train_images_pca = pca.fit_transform(train_images_flat)
test_images_pca = pca.transform(test_images_flat)
print(pca.explained_variance_ratio_.sum())

We got a variance of 85% with 40 PCA components. For PCA a higher variance allows us to distinguish classes better.

In [ ]:
# PCA plotting
plt.figure(figsize=(10, 5))
sns.scatterplot(x=train_images_pca[:, 0], y=train_images_pca[:, 1], hue=train_label_data, palette='tab10')
plt.title('PCA Visualization')
plt.xlabel('Principal Component 1')
plt.ylabel('Principal Component 2')
plt.show()

In [ ]:
# Decision Tree Classifier

dt = DecisionTreeClassifier()
dt.fit(train_images_pca, train_label_data)
predictions = dt.predict(test_images_pca)
accuracy = accuracy_score(test_label_data, predictions)
print(f'Decision Tree Classifier Accuracy: {accuracy * 100:.2f}%')

In [ ]:
# find best k
silhouette_scores = []
for k in range(2, 14):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(train_images_pca, train_label_data)
    predictions = knn.predict(test_images_pca)
    silhouette_scores.append(silhouette_score(test_images_pca, predictions))

plt.plot(range(2, 14), silhouette_scores, marker='o')
plt.xlabel('Number of Neighbors (k)')
plt.ylabel('Silhouette Score')
plt.title('KNN Silhouette Score')
plt.show()


Using the elbow method we will pick 5 as the optimal k for KNN.

In [ ]:
knn = KNeighborsClassifier(n_neighbors=k)
knn.fit(train_images_pca, train_label_data)
predictions = knn.predict(test_images_pca)
accuracy = accuracy_score(test_label_data, predictions)
print(f'KNN Classifier Accuracy: {accuracy * 100:.2f}%')

In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3),
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2))
        self.fc1 = nn.Linear(64 * 6 * 6, 512)
        self.dropout = nn.Dropout(0.5)
        self.fc2 = nn.Linear(512, 10)

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer2(x)
        x = x.view(-1, 64 * 6 * 6)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

model = CNN()
print(model)

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [ ]:
for epoch in range(10):
    model.train()
    for images, labels in train_loader:
        images = images.float()
        optimizer.zero_grad()
        output = model(images)
        loss = criterion(output, labels)
        loss.backward()
        optimizer.step()

    model.eval()
    val_loss = 0
    correct = 0
    with torch.no_grad():
        for images, labels in val_loader:
            images = images.float()
            output = model(images)
            val_loss += criterion(output, labels).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(labels.view_as(pred)).sum().item()
    val_loss /= len(val_loader.dataset)

    print('Epoch: {} \tValidation Loss: {:.6f} \tValidation Accuracy: {:.2f}%'.format(epoch + 1, val_loss, 100. * correct / len(val_loader.dataset)))

In [ ]:
# Test the model
model = CNN()
model.load_state_dict(torch.load('model.pth'))
model.eval()

test_loss = 0
correct = 0
predictions = []
with torch.no_grad():
    for images, labels in test_loader:
        images = images.float()
        output = model(images)
        test_loss += criterion(output, labels).item()
        pred = output.argmax(dim=1, keepdim=True)
        predictions.extend(pred)
        correct += pred.eq(labels.view_as(pred)).sum().item()
test_loss /= len(test_loader.dataset)

print('Test Loss: {:.6f} \tTest Accuracy: {:.2f}%'.format(test_loss, 100. * correct / len(test_loader.dataset)))

In [ ]:
# Classification Report
predictions = torch.cat(predictions, dim=0)
print(classification_report(test_label_data, predictions, target_names=[str(i) for i in range(10)]))

# Confusion Matrix
cm = confusion_matrix(test_label_data, predictions)
plt.figure(figsize=(10, 7))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=[str(i) for i in range(10)], yticklabels=[str(i) for i in range(10)])
plt.xlabel('Predicted')
plt.ylabel('Actual')

# F1 Score
f1 = f1_score(test_label_data, predictions, average='macro')
print(f'F1 Score: {f1:.2f}')

# Precision
precision = precision_score(test_label_data, predictions, average='macro')
print(f'Precision: {precision:.2f}')

# Recall
recall = recall_score(test_label_data, predictions, average='macro')
print(f'Recall: {recall:.2f}')

plt.show()

Sources:
https://pytorch.org/tutorials/beginner/blitz/cifar10_tutorial.html
https://stackoverflow.com/questions/12067446/how-many-principal-components-to-take